In [2]:
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 138 kB 9.8 MB/s 


In [3]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

In [4]:
# fix random seed for reproducibility
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(seed) # initialize a rsndom number generator 

In [5]:
# flatten 28*28 images to a 784 vector for each image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert to tensor
    transforms.Lambda(lambda x: x.view(-1))  # flatten into vector
])

In [6]:
trainset = MNIST(".", train=True, download=True, transform=transform)
testset = MNIST(".", train=False, download=True, transform=transform)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [7]:
# create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=True)

In [8]:
# define baseline model
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        if not self.training:
            out = F.softmax(out, dim=1)
        return out

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [20]:
def train_model(train_loss, train_accuracy, loss_function, optimiser, model, epoch):
  running_loss = 0.0
  accuracy = 0.0
  for data in trainloader:
    # get the inputs
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)

    # zero the parameter gradients
    optimiser.zero_grad()

    # forward + loss + backward + optimise (update weights)
    outputs = model(inputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimiser.step()

    # keep track of the loss this epoch
    running_loss += loss.item()

    model.eval()
    with torch.no_grad():
      outputs = model(inputs)
    preds = torch.max(outputs, 1)[0]
    accuracy += (preds == labels).sum()

  accuracy = accuracy/len(trainloader.dataset)
  train_accuracy[epoch] = accuracy
  # print(train_accuracy)

  running_loss = running_loss/len(trainloader)
  train_loss[epoch] = running_loss

  print("Epoch: %d, Accuracy: %4.2f,Loss: %4.2f" % (epoch, accuracy, running_loss))

  return train_accuracy, train_loss


In [19]:
def test_model(model, epoch):
  model.eval()
  with torch.no_grad():
    running_loss = 0.0
    accuracy = 0.0
    for data in testloader:
      # get the inputs
      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)
      preds = torch.max(outputs, 1)[0]
      accuracy += (preds == labels).sum()

  accuracy = accuracy/len(testloader.dataset)
  test_accuracy[epoch] = accuracy

  running_loss = running_loss/len(testloader)
  test_loss[epoch] = running_loss

  #print("Epoch: %d, Accuracy: %4.2f,Loss: %4.2f" % (epoch, accuracy, running_loss))
  return test_accuracy, test_loss


In [21]:
epoch = 20
# build the model 
model = BaselineModel(784, 500, 10).to(device)

# define the loss function and the optimiser
loss_function = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters())

train_loss = torch.empty(epoch).to(device)  
train_accuracy = torch.empty(epoch).to(device)

test_loss = torch.empty(epoch).to(device)  
test_accuracy = torch.empty(epoch).to(device)

for epoch in range(1, epoch+1):
  print("Training Dataset ")
  # the epoch loop
  train_model(train_loss, train_accuracy, loss_function, optimiser, model, epoch)
  print("**Finished Training**")

  print("Testing Dataset ")
  test_model(model, epoch)
  print("**Finished Testing**")


Training Dataset 
Epoch: 1, Accuracy: 0.00,Loss: 1.60
**Finished Training**
Testing Dataset 
**Finished Testing**
Training Dataset 
Epoch: 2, Accuracy: 0.00,Loss: 1.52
**Finished Training**
Testing Dataset 
**Finished Testing**
Training Dataset 


KeyboardInterrupt: ignored